In [1]:
#!pip3 install pymongo
#!pip3 install pandas
#!pip3 install h5py

from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

import string
from nltk.corpus import stopwords

Using TensorFlow backend.


In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth',-1) # ini untuk melihat full column
client = MongoClient('localhost', 27017) #ihi host
db=client.get_database('fujiatma') # ini nama database sebelumnya create dlu di mongodb atau di robomongo
collection=db.get_collection('data')# ini nama collectionnya (misal data)
#collection.find_one({ "question" : "Does it fit Nook GlowLight?"})# ini contoh query saja


#cara dump ke mongo
#mongoimport --db fujiatma --collection data --file intents.json

cursor=collection.find() # query untuk melihat isi collection
data = pd.DataFrame(list(collection.find())) # merubah ke data frame
#data=data[data['question']!=np.nan]
#data=data.dropna(axis=0, how='all')
#data.head()

data_ques_answer=data[['question','answer']][:100] #saya ambil cuma 100 data
print ("banyak data:", len(data_ques_answer))
data_ques_answer.head()

banyak data: 100


,question,answer
0,Is this cover the one that fits the old nook color? Which I believe is 8x5.,Yes this fits both the nook color and the same-shaped nook tablet
1,Does it fit Nook GlowLight?,No. The nook color or color tablet
2,Would it fit Nook 1st Edition? 4.9in x 7.7in ?,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close."
3,Will this fit a Nook Color that's 5 x 8?,yes
4,will this fit the Samsung Galaxy Tab 4 Nook 10.1,"No, the tab is smaller than the 'color'"


In [3]:
def preprocess(corpus):
    #melakukan proses lowercase, stopword removal, dan punctuation removal
    print ("proses lowercase")
    m=[]
    
    corpus['question']=corpus.question.astype(str).str.lower()
    print ("proses stopwords removal")
    cachedStopWords = set(stopwords.words("english"))
    for i in range(len(corpus)):
        sent=corpus['question'].iloc[i]
        kt=" ".join([word for word in sent.split() if word not in cachedStopWords])
        corpus['question'].iloc[i]=kt
        
    print ("proses punctuation")
    remove=string.punctuation

    for i in range(len(corpus)):
        sent=data_ques_answer['question'].iloc[i]
        kd=' '.join(word.strip(remove) for word in sent.split())
        corpus['question'].iloc[i]=kd
    return corpus


#if types of column is str, astype is not necessary
#data.question = data.question.astype(str).str.lower()
#df = df.astype(str).apply(lambda x: x.str.lower())
#data

data=preprocess(data_ques_answer)
data

proses lowercase
proses stopwords removal
proses punctuation


,question,answer
0,cover one fits old nook color believe 8x5,Yes this fits both the nook color and the same-shaped nook tablet
1,fit nook glowlight,No. The nook color or color tablet
2,would fit nook 1st edition 4.9in x 7.7in,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close."
3,fit nook color that's 5 x 8,yes
4,fit samsung galaxy tab 4 nook 10.1,"No, the tab is smaller than the 'color'"
5,flip stand,"No, there is not a flip stand. It has a pocket in the front flap. It is a very nice cover."
6,flip stand,"Hi, no it doesn't"
7,also fits hd,It should. They are the same size and the charging port is in the same place.
8,2 positions reader horizontal/vertical thank kwod,Yes
9,closure mechanism bands magnetic etc,No- it is more like a normal book would be. It doesn't flop open- so it has never wen an issue for me. The nook clips into a secure and safe holder inside that is small and convenient. This is the best cover I've ever had for my nook- trim and functional (no magnets or elastic needed).


In [4]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding

In [5]:
X = data.question
Y = data.answer
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [7]:
max_words = 1000 #
max_len = 10
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [8]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [9]:
from keras.optimizers import RMSprop
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 50)            50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [10]:
from keras.callbacks import EarlyStopping
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=100,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])


Train on 68 samples, validate on 17 samples
Epoch 1/100
68/68 [==============================] - 0s - loss: 0.6858 - acc: 0.0294 - val_loss: -1.6298 - val_acc: 0.0000e+00
Epoch 2/100
68/68 [==============================] - 0s - loss: -2.3147 - acc: 0.0147 - val_loss: -4.6574 - val_acc: 0.0000e+00
Epoch 3/100
68/68 [==============================] - 0s - loss: -6.2815 - acc: 0.0147 - val_loss: -9.1965 - val_acc: 0.0000e+00
Epoch 4/100
68/68 [==============================] - 0s - loss: -12.2494 - acc: 0.0147 - val_loss: -16.9219 - val_acc: 0.0000e+00
Epoch 5/100
68/68 [==============================] - 0s - loss: -22.7263 - acc: 0.0147 - val_loss: -30.7716 - val_acc: 0.0000e+00
Epoch 6/100
68/68 [==============================] - 0s - loss: -40.7140 - acc: 0.0147 - val_loss: -56.6239 - val_acc: 0.0000e+00
Epoch 7/100
68/68 [==============================] - 0s - loss: -75.0172 - acc: 0.0147 - val_loss: -106.3685 - val_acc: 0.0000e+00
Epoch 8/100
68/68 [==============================] -

In [11]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

15/15 [==============================] - 0s
Test set
  Loss: -698.276
  Accuracy: 0.000
